In [1]:
!pip install --quiet datasets nltk sumy evaluate rouge-score

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 202

**Frequency Based Method with rouge score**

In [10]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import string
from datasets import load_dataset
import evaluate

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

# Load the ROUGE metric
rouge = evaluate.load('rouge')

# Frequency-based summarization function
def frequency_based_summarization(text):
    words = word_tokenize(text.lower())
    sentences = sent_tokenize(text)

    stop_words = set(stopwords.words("english"))
    words = [word for word in words if word not in stop_words and word not in string.punctuation]

    word_freq = {}
    for word in words:
        if word in word_freq:
            word_freq[word] += 1
        else:
            word_freq[word] = 1

    max_freq = max(word_freq.values())
    for word in word_freq:
        word_freq[word] = word_freq[word] / max_freq

    sentence_scores = {}
    for sentence in sentences:
        sentence_words = word_tokenize(sentence.lower())
        for word in sentence_words:
            if word in word_freq:
                if sentence not in sentence_scores:
                    sentence_scores[sentence] = word_freq[word]
                else:
                    sentence_scores[sentence] += word_freq[word]

    sorted_sentences = sorted(sentence_scores, key=sentence_scores.get, reverse=True)
    summary_sentences = sorted_sentences[:3]

    summary = ' '.join(summary_sentences)
    return summary

# Function to compute ROUGE score
def compute_rouge(predictions, references):
    rouge_output = rouge.compute(predictions=predictions, references=references, use_stemmer=True)
    return rouge_output

# Load CNN/Daily Mail dataset
cnn_dataset = load_dataset('cnn_dailymail', '3.0.0', split='test[:20%]')

# Select a subset of the dataset for summarization
source_texts = cnn_dataset['article'][:100]
reference_summaries = cnn_dataset['highlights'][:100]

# Generate summaries using the frequency-based method
frequency_summaries = [frequency_based_summarization(text) for text in source_texts]

# Calculate ROUGE scores
rouge_scores = compute_rouge(frequency_summaries, reference_summaries)

# Print one example summary and ROUGE scores
print("Original Text:")
print(source_texts[0])

print("\nFrequency-based Summary:")
print(frequency_summaries[0])

print("\nReference Summary:")
print(reference_summaries[0])

print("\nROUGE Scores:")
print(rouge_scores)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Original Text:
(CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC's founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki, speaking at Wednesday

**Luhn Method Rouge Score**

In [4]:
# Import necessary libraries
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.luhn import LuhnSummarizer
from datasets import load_dataset
import evaluate

rouge = evaluate.load('rouge')

# Luhn summarization method
def luhn_method(text, sentences_count=2):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer_luhn = LuhnSummarizer()

    summary = summarizer_luhn(parser.document, sentences_count)
    summary_sentences = [str(sentence) for sentence in summary]

    return ' '.join(summary_sentences)

# Function to compute ROUGE score
def compute_rouge(predictions, references):
    rouge_output = rouge.compute(predictions=predictions, references=references, use_stemmer=True)
    return rouge_output

dataset = load_dataset('cnn_dailymail', '3.0.0', split='test[:20%]')

source_texts = dataset['article'][:100]
reference_summaries = dataset['highlights'][:100]

luhn_summaries = [luhn_method(text) for text in source_texts]

rouge_scores = compute_rouge(luhn_summaries, reference_summaries)

print("Luhn Summary:")
print(luhn_summaries[0])

print("\nReference Summary:")
print(reference_summaries[0])

print("\nROUGE Scores:")
print(rouge_scores)


Luhn Summary:
(CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. "As we have said repeatedly, we do not believe that Palestine is a state and therefore we do not believe that it is eligible to join the ICC," the State Department said in a statement.

Reference Summary:
Membership gives the ICC jurisdiction over alleged crimes committed in Palestinian territories since last June .
Israel and the United States opposed the move, which could open the door to war crimes investigations against Israelis .

ROUGE Scores:
{'rouge1': 0.2720598535800665, 'rouge2': 0.08883504209760014, 'rougeL': 0.18713133013118516, 'rougeLsum': 0.21829016884073468}


**LSA method Rouge Score**

In [9]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
from datasets import load_dataset
import nltk
import evaluate

nltk.download('punkt')

rouge = evaluate.load('rouge')

# Function for LSA-based summarization
def lsa_summarization(text, sentence_count=2):
    # Create a PlaintextParser and tokenize the text
    parser = PlaintextParser.from_string(text, Tokenizer("english"))

    summarizer = LsaSummarizer()

    summary = summarizer(parser.document, sentence_count)

    summary_sentences = [str(sentence) for sentence in summary]

    final_summary = " ".join(summary_sentences)

    return final_summary

# Function to compute ROUGE score
def compute_rouge(predictions, references):
    rouge_output = rouge.compute(predictions=predictions, references=references, use_stemmer=True)
    return rouge_output

cnn_dataset = load_dataset('cnn_dailymail', '3.0.0', split='test[:20%]')

source_texts = cnn_dataset['article'][:100]
reference_summaries = cnn_dataset['highlights'][:100]

lsa_summaries = [lsa_summarization(text, sentence_count=3) for text in source_texts]

rouge_scores = compute_rouge(lsa_summaries, reference_summaries)

print("source Text:")
print(source_texts[0])

print("LSA Summary:")
print(lsa_summaries[0])

print("\nReference Summary:")
print(reference_summaries[0])
print("\nROUGE Scores:")
print(rouge_scores)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


source Text:
(CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC's founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki, speaking at Wednesday's

**Lexrank Rouge Score**

In [11]:
# Import necessary libraries
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from datasets import load_dataset
import evaluate

# Load the ROUGE metric
rouge = evaluate.load('rouge')

# LexRank summarization method
def lexrank_method(text, sentences_count=2):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer_lexrank = LexRankSummarizer()

    summary = summarizer_lexrank(parser.document, sentences_count)
    summary_sentences = [str(sentence) for sentence in summary]

    return ' '.join(summary_sentences)

# Function to compute ROUGE score
def compute_rouge(predictions, references):
    rouge_output = rouge.compute(predictions=predictions, references=references, use_stemmer=True)
    return rouge_output

# Load CNN/Daily Mail dataset
cnn_dataset = load_dataset('cnn_dailymail', '3.0.0', split='test[:20%]')

# Select a subset of the dataset for summarization
source_texts = cnn_dataset['article'][:100]
reference_summaries = cnn_dataset['highlights'][:100]

# Generate summaries using the LexRank method
lexrank_summaries = [lexrank_method(text) for text in source_texts]

# Calculate ROUGE scores
rouge_scores = compute_rouge(lexrank_summaries, reference_summaries)

# Print one example summary and ROUGE scores
print("Original Text:")
print(source_texts[0])

print("\nLexRank Summary:")
print(lexrank_summaries[0])

print("\nReference Summary:")
print(reference_summaries[0])

print("\nROUGE Scores:")
print(rouge_scores)


Original Text:
(CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories. The formal accession was marked with a ceremony at The Hague, in the Netherlands, where the court is based. The Palestinians signed the ICC's founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014." Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis. As members of the court, Palestinians may be subject to counter-charges as well. Israel and the United States, neither of which is an ICC member, opposed the Palestinians' efforts to join the body. But Palestinian Foreign Minister Riad al-Malki, speaking at Wednesday